<a href="https://colab.research.google.com/github/srinijammula/Infringing_URL/blob/main/Infringing_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and imports

In [ ]:
import pandas as pd
import json

In [ ]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade python-whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install plotly

### Data

In [ ]:
#read data from file
with open('response.json', 'r') as file:
    data = json.load(file)

In [ ]:
#construct each row based on infringing_urls
flattened_data = []

for notice in data['notices']:
    #do not include works column in base_info as we add it later for infringing_url
    base_info={key:notice.get(key,None) for key in notice.keys() if key!='works'}
    #these columns needs to be flattened further as they contain data in arrays
    for i in ('topics','tags','jurisdictions'):
      base_info[i]=','.join(base_info[i])
    for work in notice.get('works',[]):
        work_info=base_info.copy()
        work_info['description']=work.get('description',None)
        for url in work.get('infringing_urls',[]):
            row=work_info.copy()
            row['infringing_url']=url.get('url',None)
            flattened_data.append(row)


In [ ]:
#add domain and ip for url
import urllib.parse
import socket

def process_cpu(t):
  url=t['infringing_url']
  parsed_url=urllib.parse.urlparse(url)
  domain=parsed_url.netloc
  try:
    ip=socket.gethostbyname(domain)
  except socket.error:
    ip=None
  t['domain']=domain
  t['ip_address']=ip
  return t

In [ ]:
#parallelization using 4 cpus
from multiprocessing import Pool

with Pool(processes=4) as pool:
  updated_data=pool.map(process_cpu,flattened_data)

In [ ]:
#create a data frame from final data
df=pd.DataFrame(updated_data)
df

,id,type,title,body,date_sent,date_received,topics,sender_name,principal_name,recipient_name,tags,jurisdictions,action_taken,language,score,description,infringing_url,domain,ip_address
0,23878572,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-24T00:00:00.000Z,2021-05-24T00:00:00.000Z,"DMCA Notices,Copyright",3Ants Development & Strategies S.L.,Netflix,Google LLC,,ES,None,None,None,FILM - El baile de los *41 - Dance of the Fort...,https://www.poseidonhd.in/pelicula/ver-online-...,www.poseidonhd.in,95.215.19.22
1,23878572,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-24T00:00:00.000Z,2021-05-24T00:00:00.000Z,"DMCA Notices,Copyright",3Ants Development & Strategies S.L.,Netflix,Google LLC,,ES,None,None,None,FILM - El baile de los *41 - Dance of the Fort...,https://www1.cuevana3.video/13445/el-baile-de-...,www1.cuevana3.video,104.21.8.94
2,23878572,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-24T00:00:00.000Z,2021-05-24T00:00:00.000Z,"DMCA Notices,Copyright",3Ants Development & Strategies S.L.,Netflix,Google LLC,,ES,None,None,None,FILM - El baile de los *41 - Dance of the Fort...,https://pelisplus.live/el-baile-de-los-41-2021...,pelisplus.live,77.37.89.135
3,23878572,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-24T00:00:00.000Z,2021-05-24T00:00:00.000Z,"DMCA Notices,Copyright",3Ants Development & Strategies S.L.,Netflix,Google LLC,,ES,None,None,None,FILM - El baile de los *41 - Dance of the Fort...,https://pelis28.nu/ver-pelicula/el-baile-de-lo...,pelis28.nu,104.247.81.51
4,23878572,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-24T00:00:00.000Z,2021-05-24T00:00:00.000Z,"DMCA Notices,Copyright",3Ants Development & Strategies S.L.,Netflix,Google LLC,,ES,None,None,None,FILM - El baile de los *41 - Dance of the Fort...,https://pelis24.app/ver-el-baile-de-los-41-onl...,pelis24.app,172.67.198.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66164,23674110,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-03T00:00:00.000Z,2021-05-03T00:00:00.000Z,"DMCA Notices,Copyright",Vobile Inc,Netflix,Google LLC,,us,None,None,None,Castlevania: Season 3 Episode 5: A Seat of Civ...,https://chomikuj.pl/aleserialiki/Galeria/Zacho...,chomikuj.pl,104.22.25.70
66165,23674110,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-03T00:00:00.000Z,2021-05-03T00:00:00.000Z,"DMCA Notices,Copyright",Vobile Inc,Netflix,Google LLC,,us,None,None,None,Castlevania: Season 2 Episode 2: Old Homes\n,https://chomikuj.pl/aleserialiki/SERIALE/Castl...,chomikuj.pl,104.22.25.70
66166,23674110,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-03T00:00:00.000Z,2021-05-03T00:00:00.000Z,"DMCA Notices,Copyright",Vobile Inc,Netflix,Google LLC,,us,None,None,None,Castlevania: Season 2 Episode 2: Old Homes\n,https://chomikuj.pl/aleserialiki/Galeria/Zacho...,chomikuj.pl,104.22.25.70
66167,23674110,DMCA,DMCA (Copyright) Complaint to Google,None,2021-05-03T00:00:00.000Z,2021-05-03T00:00:00.000Z,"DMCA Notices,Copyright",Vobile Inc,Netflix,Google LLC,,us,None,None,None,Castlevania: Season 3 Episode 1: Bless Your De...,https://chomikuj.pl/aleserialiki/SERIALE/Castl...,chomikuj.pl,104.22.25.70


In [ ]:
#download a csv from data frame
df.to_csv('data.csv')

In [ ]:
#column name and their unique values count
for i in df:
  print(i,'-',len(df[i].unique()))
  print('---------------------------')
  #print(df[i].unique())

id - 500
---------------------------
type - 1
---------------------------
title - 1
---------------------------
body - 1
---------------------------
date_sent - 22
---------------------------
date_received - 22
---------------------------
topics - 1
---------------------------
sender_name - 13
---------------------------
principal_name - 4
---------------------------
recipient_name - 1
---------------------------
tags - 1
---------------------------
jurisdictions - 9
---------------------------
action_taken - 1
---------------------------
language - 1
---------------------------
score - 1
---------------------------
description - 775
---------------------------
infringing_url - 66042
---------------------------
domain - 3010
---------------------------
ip_address - 1178
---------------------------


### Summary 1 - Domain & IP Analysis



*   There are total 66,042 unique infringing urls but only 3010 unique domains and 1267 unique IP addresses. This means multiple infringing urls are hosted on same domain.


*   Let's list the top 10 domains and ips used to host these infringing urls, with corresponding count of infringing urls. This can be used to take enforcement actions on the domains with more urls with priority.

*   For further analysis, whois lookup is used to list service providers for domains and hostname of IP addresses.







In [ ]:
from collections import Counter

#dictionary to store domain and ip with count of infringing urls
domain_counts=Counter(t['domain'] for t in updated_data)
ip_counts=Counter(t['ip_address'] for t in updated_data)
#select top 10
top_domains=domain_counts.most_common(10)
top_ips=ip_counts.most_common(10)

print("Top Domains Hosting Infringing Content:")
print('-----------------------------------------')
for domain,count in top_domains:
    print(f"Domain: {domain},Count: {count}")

print("\nTop IPs Hosting Infringing Content:")
print('-----------------------------------------')
for ip,count in top_ips:
    print(f"IP: {ip}, Count: {count}")

Top Domains Hosting Infringing Content:
-----------------------------------------
Domain: chomikuj.pl,Count: 23605
Domain: watchepisodeseries.unblockit.onl,Count: 5422
Domain: rapidgator.net,Count: 1760
Domain: www.torlock.cc,Count: 932
Domain: ul.to,Count: 851
Domain: drive.google.com,Count: 730
Domain: www.filefactory.com,Count: 701
Domain: ok.ru,Count: 662
Domain: vidlox.me,Count: 614
Domain: 1337x.mrunblock.surf,Count: 555

Top IPs Hosting Infringing Content:
-----------------------------------------
IP: 172.67.42.83, Count: 16315
IP: 104.22.25.70, Count: 7290
IP: 172.234.222.138, Count: 7113
IP: None, Count: 3262
IP: 195.211.222.2, Count: 1760
IP: 172.234.222.143, Count: 1384
IP: 172.232.4.213, Count: 1244
IP: 103.224.182.253, Count: 1169
IP: 199.115.116.216, Count: 1151
IP: 199.59.243.226, Count: 1096


In [ ]:
#list the service provider of each domain
import whois

for domain,count in top_domains:
      w=whois.whois(domain)
      print(f"Domain: {domain}, Registrar: {w.registrar}, Country: {w.country}")

Domain: chomikuj.pl, Registrar: Instra Corporation Pty Ltd, Country: None
Domain: watchepisodeseries.unblockit.onl, Registrar: NAMECHEAP INC, Country: IS
Domain: rapidgator.net, Registrar: Internet Domain Service BS Corp., Country: BS
Domain: www.torlock.cc, Registrar: NameSilo, LLC, Country: US


2024-08-18 23:04:47,013 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 111] Connection refused
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 111] Connection refused


Domain: ul.to, Registrar: None, Country: None
Domain: drive.google.com, Registrar: MarkMonitor, Inc., Country: US
Domain: www.filefactory.com, Registrar: GoDaddy.com, LLC, Country: None
Domain: ok.ru, Registrar: RU-CENTER-RU, Country: None
Domain: vidlox.me, Registrar: NAMECHEAP INC, Country: None
Domain: 1337x.mrunblock.surf, Registrar: DYNADOT LLC, Country: US


In [ ]:
#hostanme based on IP address
for ip,count in top_ips:
    try:
        host=socket.gethostbyaddr(ip)
        print(f"IP: {ip}, Hostname: {host[0]}")
    except:
        print(f"IP: {ip}, Error")

IP: 172.67.42.83, Error
IP: 104.22.25.70, Error
IP: 172.234.222.138, Hostname: 172-234-222-138.ip.linodeusercontent.com
IP: None, Error
IP: 195.211.222.2, Error
IP: 172.234.222.143, Hostname: 172-234-222-143.ip.linodeusercontent.com
IP: 172.232.4.213, Hostname: 172-232-4-213.ip.linodeusercontent.com
IP: 103.224.182.253, Hostname: lb-182-253.above.com
IP: 199.115.116.216, Error
IP: 199.59.243.226, Error


### Summary 2 - Temporal Analysis

*   Analyze the data based on notice sent date and recieved date.
*   Comapring unique and sorted values from fields date_sent and date_received, gives same values, so I decided to analyze on one which is date_sent.
*   Created a live dashboard using plotly based on date_sent and notices received. If we point on the highest spike, we can see it is on May 15, 2021 with 9367 notices sent.




In [ ]:
#unique and sorted values of date sent and date received
print('Date Sent \t\t\t Date recieved')
for i,j in zip(sorted(df['date_sent'].unique()),sorted(df['date_received'].unique())):
  print(i,'\t',j)

Date Sent 			 Date recieved
2021-05-03T00:00:00.000Z 	 2021-05-03T00:00:00.000Z
2021-05-04T00:00:00.000Z 	 2021-05-04T00:00:00.000Z
2021-05-05T00:00:00.000Z 	 2021-05-05T00:00:00.000Z
2021-05-06T00:00:00.000Z 	 2021-05-06T00:00:00.000Z
2021-05-07T00:00:00.000Z 	 2021-05-07T00:00:00.000Z
2021-05-08T00:00:00.000Z 	 2021-05-08T00:00:00.000Z
2021-05-09T00:00:00.000Z 	 2021-05-09T00:00:00.000Z
2021-05-10T00:00:00.000Z 	 2021-05-10T00:00:00.000Z
2021-05-11T00:00:00.000Z 	 2021-05-11T00:00:00.000Z
2021-05-12T00:00:00.000Z 	 2021-05-12T00:00:00.000Z
2021-05-13T00:00:00.000Z 	 2021-05-13T00:00:00.000Z
2021-05-14T00:00:00.000Z 	 2021-05-14T00:00:00.000Z
2021-05-15T00:00:00.000Z 	 2021-05-15T00:00:00.000Z
2021-05-16T00:00:00.000Z 	 2021-05-16T00:00:00.000Z
2021-05-17T00:00:00.000Z 	 2021-05-17T00:00:00.000Z
2021-05-18T00:00:00.000Z 	 2021-05-18T00:00:00.000Z
2021-05-19T00:00:00.000Z 	 2021-05-19T00:00:00.000Z
2021-05-20T00:00:00.000Z 	 2021-05-20T00:00:00.000Z
2021-05-21T00:00:00.000Z 	 2021-05-2

In [ ]:
#plot date sent against no. of notices sent on that day
import pandas as pd
import plotly.express as px

temp_df={}
temp_df['date_sent']=pd.to_datetime(df['date_sent']).dt.date
daily_sent_counts=temp_df['date_sent'].value_counts().sort_index()
sent_df=pd.DataFrame({'Date': daily_sent_counts.index,'Count': daily_sent_counts.values})

fig_sent=px.line(sent_df,x='Date',y='Count',title='Number of Notices Sent Per Day',labels={'Date':'Date Sent','Count':'Number of Notices Sent'})
fig_sent.show()

### Summary 3 - Regional Analysis


*   In the data we saw in total there are only 9 unique jursidictions, so we can see the geographical trends of notices sent.


*   Mapping the country code to name for readability.


*   Plotting a live bar graph with jursidications on x-axis and no. of notices on y-axis. We can see that United states and India have issued majority of notices.







In [ ]:
print('Jurisdictions')
for i in df['jurisdictions'].unique():
  print(i)

Jurisdictions
ES
IN
us
KR
vn
KZ
RU
MY
ID


In [ ]:
#mapping country code to name
jurisdiction_map={'ES':'Spain','IN':'India','us':'United States','KR':'South Korea','vn':'Vietnam','KZ':'Kazakhstan','RU':'Russia','MY':'Malaysia','ID':'Indonesia'}

In [ ]:
#plot
jurisdiction_counts=df['jurisdictions'].map(jurisdiction_map).value_counts()
jurisdiction_df=pd.DataFrame({'Jurisdiction':jurisdiction_counts.index,'Count':jurisdiction_counts.values})

fig = px.bar(jurisdiction_df,x='Jurisdiction',y='Count',title='Number of Notices by Jurisdiction',labels={'Jurisdiction':'Jurisdiction','Count':'Number of Notices'},text='Count')
fig.show()